In [1]:
from onnx2keras import onnx_to_keras
import torch
import onnx
import io
import logging
# with tf.device('/cpu:0'):

In [2]:
import os
import sys
import glob;
import math;
import numpy as np;
import glob;
import random;
import time;

In [68]:
import keras_core as keras
import tensorflow as tf
from onnx_tf.backend import prepare

In [4]:
if torch.cuda.is_available():
    DEFAULT_DEVICE = "cuda"
elif torch.backends.mps.is_available():
    DEFAULT_DEVICE = "mps"
else:
    DEFAULT_DEVICE = "cpu"

In [5]:
import onnxruntime as rt

In [6]:
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [7]:
print(DEFAULT_DEVICE)

mps


In [8]:
# import torch;
import torch.optim as optim;
import torch.nn as nn;
from torch.autograd import Variable
sys.path.append(os.getcwd());
sys.path.append(os.path.abspath('../'));
# sys.path.append(os.path.join(os.getcwd(), 'torch/resources'));
import common.utils as U;
import common.opts as opts;
# import resources.models as models;
import th.resources.calculator as calc;
import common.tlopts as tlopts
# import resources.train_generator as train_generator;
import argparse
from itertools import repeat

In [9]:
#Reproducibility
seed = 42;
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
torch.backends.cudnn.deterministic = True;
torch.backends.cudnn.benchmark = False;

In [10]:
pruned_and_retrained_acdnet = './retrained_models_after_pruned/retrained_model_20240124123209_acc_95.45455169677734_795th_epoch.pt'

In [30]:
class ACDNetV2(nn.Module):
    def __init__(self, input_length, n_class, sr, ch_conf=None):
        super(ACDNetV2, self).__init__();
        self.input_length = input_length;
        self.ch_config = ch_conf;

        stride1 = 2;
        stride2 = 2;
        channels = 8;
        k_size = (3, 3);
        n_frames = (sr/1000)*10; #No of frames per 10ms

        sfeb_pool_size = int(n_frames/(stride1*stride2));
        # tfeb_pool_size = (2,2);
        if self.ch_config is None:
            self.ch_config = [channels, channels*8, channels*4, channels*8, channels*8, channels*16, channels*16, channels*32, channels*32, channels*64, channels*64, n_class];
        # avg_pool_kernel_size = (1,4) if self.ch_config[1] < 64 else (2,4);
        fcn_no_of_inputs = self.ch_config[-1];
        conv1, bn1 = self.make_layers(1, self.ch_config[0], (1, 9), (1, stride1));
        conv2, bn2 = self.make_layers(self.ch_config[0], self.ch_config[1], (1, 5), (1, stride2));
        conv3, bn3 = self.make_layers(1, self.ch_config[2], k_size, padding=1);
        conv4, bn4 = self.make_layers(self.ch_config[2], self.ch_config[3], k_size, padding=1);
        conv5, bn5 = self.make_layers(self.ch_config[3], self.ch_config[4], k_size, padding=1);
        conv6, bn6 = self.make_layers(self.ch_config[4], self.ch_config[5], k_size, padding=1);
        conv7, bn7 = self.make_layers(self.ch_config[5], self.ch_config[6], k_size, padding=1);
        conv8, bn8 = self.make_layers(self.ch_config[6], self.ch_config[7], k_size, padding=1);
        conv9, bn9 = self.make_layers(self.ch_config[7], self.ch_config[8], k_size, padding=1);
        conv10, bn10 = self.make_layers(self.ch_config[8], self.ch_config[9], k_size, padding=1);
        conv11, bn11 = self.make_layers(self.ch_config[9], self.ch_config[10], k_size, padding=1);
        conv12, bn12 = self.make_layers(self.ch_config[10], self.ch_config[11], (1, 1));
        fcn = nn.Linear(fcn_no_of_inputs, n_class);
        nn.init.kaiming_normal_(fcn.weight, nonlinearity='sigmoid') # kaiming with sigoid is equivalent to lecun_normal in keras

        self.sfeb = nn.Sequential(
            #Start: Filter bank
            conv1, bn1, nn.ReLU(),\
            conv2, bn2, nn.ReLU(),\
            nn.MaxPool2d(kernel_size=(1, sfeb_pool_size))
        );

        tfeb_modules = [];
        self.tfeb_width = int(((self.input_length / sr)*1000)/10); # 10ms frames of audio length in seconds
        tfeb_pool_sizes = self.get_tfeb_pool_sizes(self.ch_config[1], self.tfeb_width);
        p_index = 0;
        for i in [3,4,6,8,10]:
            tfeb_modules.extend([eval('conv{}'.format(i)), eval('bn{}'.format(i)), nn.ReLU()]);

            if i != 3:
                tfeb_modules.extend([eval('conv{}'.format(i+1)), eval('bn{}'.format(i+1)), nn.ReLU()]);

            h, w = tfeb_pool_sizes[p_index];
            if h>1 or w>1:
                tfeb_modules.append(nn.MaxPool2d(kernel_size = (h,w)));
            p_index += 1;

        tfeb_modules.append(nn.Dropout(0.2));
        tfeb_modules.extend([conv12, bn12, nn.ReLU()]);
        h, w = tfeb_pool_sizes[-1];
        if h>1 or w>1:
            tfeb_modules.append(nn.AvgPool2d(kernel_size = (h,w)));
        tfeb_modules.extend([nn.Flatten(), fcn]);

        self.tfeb = nn.Sequential(*tfeb_modules);

        self.output = nn.Sequential(
            nn.Softmax(dim=1)
        );

    def forward(self, x):
        x = self.sfeb(x);
        #swapaxes
        x = x.permute((0, 2, 1, 3));
        x = self.tfeb(x);
        y = self.output[0](x);
        return y;

    def make_layers(self, in_channels, out_channels, kernel_size, stride=(1,1), padding=0, bias=False):
        conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias);
        nn.init.kaiming_normal_(conv.weight, nonlinearity='relu'); # kaiming with relu is equivalent to he_normal in keras
        bn = nn.BatchNorm2d(out_channels);
        return conv, bn;

    def get_tfeb_pool_sizes(self, con2_ch, width):
        h = self.get_tfeb_pool_size_component(con2_ch);
        w = self.get_tfeb_pool_size_component(width);
        # print(w);
        pool_size = [];
        for  (h1, w1) in zip(h, w):
            pool_size.append((h1, w1));
        return pool_size;

    def get_tfeb_pool_size_component(self, length):
        # print(length);
        c = [];
        index = 1;
        while index <= 6:
            if length >= 2:
                if index == 6:
                    c.append(length);
                else:
                    c.append(2);
                    length = length // 2;
            else:
               c.append(1);

            index += 1;

        return c;

def GetACDNetModel(input_len=30225, nclass=2, sr=20000, channel_config=None):
    net = ACDNetV2(input_len, nclass, sr, ch_conf=channel_config);
    return net;

In [31]:
def getPruned_and_Retrained_Model(pruned_and_retrain_model_path):
    state = torch.load(pruned_and_retrain_model_path, map_location=torch.device('cpu'));
    weights = state['weight']
    config = state['config']
    net = GetACDNetModel(channel_config=config)
    print(f"config is {config}")
    net.load_state_dict(weights)
    return net

In [32]:
retrain_model = getPruned_and_Retrained_Model(pruned_and_retrain_model_path=pruned_and_retrained_acdnet)

config is [5, 32, 9, 16, 23, 33, 29, 56, 47, 65, 90, 2]


In [33]:
#show the retrain_model structure
calc.summary(retrain_model, (1,1,30225));

+----------------------------------------------------------------------------+
+                           Pytorch Model Summary                            +
------------------------------------------------------------------------------
   Layer (type)       Input Shape      Output Shape    Param #      FLOPS #
       Conv2d-1     (1, 1, 30225)     (5, 1, 15109)         45      679,905
  BatchNorm2d-2     (5, 1, 15109)     (5, 1, 15109)         10            0
         ReLu-3     (5, 1, 15109)     (5, 1, 15109)          0       75,545
       Conv2d-4     (5, 1, 15109)     (32, 1, 7553)        800    6,042,400
  BatchNorm2d-5     (32, 1, 7553)     (32, 1, 7553)         64            0
         ReLu-6     (32, 1, 7553)     (32, 1, 7553)          0      241,696
    MaxPool2d-7     (32, 1, 7553)      (32, 1, 151)          0      241,600
      Permute-8      (32, 1, 151)      (1, 32, 151)          0            0
       Conv2d-9      (1, 32, 151)      (9, 32, 151)         81      391,392
 Ba

In [34]:
input_np = np.random.uniform(0, 1, (1, 1, 1, 30225))
input_var = Variable(torch.FloatTensor(input_np))
output = retrain_model(input_var)

In [35]:
print(input_var)
print(output)

tensor([[[[0.3238, 0.8499, 0.7078,  ..., 0.5673, 0.7980, 0.4644]]]])
tensor([[0.6254, 0.3746]], grad_fn=<SoftmaxBackward0>)


## first convert to onnx

In [17]:
"""
x = torch.randn(1, 1, 256, 784, requires_grad=True)
torch.onnx.export(model, x, "torchToOnnx.onnx", verbose=True, input_names = ['input'], output_names = ['output'])
"""
# x =  torch.randn(1,1,1,30225, requires_grad=True)
# torch.onnx.export(retrain_model,x,"./onnx_models/retrain_pt2onnx.onnx", verbose=True, input_names = ['input'], output_names = ['output'])

'\nx = torch.randn(1, 1, 256, 784, requires_grad=True)\ntorch.onnx.export(model, x, "torchToOnnx.onnx", verbose=True, input_names = [\'input\'], output_names = [\'output\'])\n'

In [18]:
sess = rt.InferenceSession("./onnx_models/retrain_pt2onnx.onnx")
input_name = sess.get_inputs()[0].name
print(input_name)
print(type(input_np))
# Note: The input must be of the same shape as the shape of x during # the model export part. i.e. second argument in this function call: torch.onnx.export()
onnxPredictions = sess.run(None, {input_name: np.float32(input_np)})[0]
print(onnxPredictions)

input
<class 'numpy.ndarray'>
[[0.54944634 0.45055372]]


## onnx to keras
ref:
https://github.com/PINTO0309/onnx2tf


In [66]:
# Load ONNX model
onnx_model = onnx.load('./onnx_models/retrain_pt2onnx.onnx')
# Call the converter (input will be equal to the input_names parameter that you defined during exporting)
# k_model = onnx_to_keras(onnx_model, ['input'], name_policy='renumerate')
# pytorch_to_keras(model, input_var, [(3, 224, 224,)], verbose=True, change_ordering=True)

In [67]:
# !pip install tensorflow-probability

In [86]:
# tf_rep = prepare(onnx_model)  # prepare tf representation
# tf_rep.export_graph("path")
# print(tf_rep.graph)

In [44]:
# k_model.summary()

In [72]:
# k_in = np.transpose(input_np, [0,2,3,1])
# print(k_in.shape)
# k_model.d
# k_model.predict(k_in)


# k_input = np.transpose(input_np, [0, 3, 1, 2])
# k_model.predict(k_input,data_format="NCHW")

In [32]:
# k_model.save("./keras_models/retrain_model.keras")

In [54]:
# tf.ConfigProto(log_device_placement=True)

In [87]:
# tfmodel = tf.saved_model.load("./tensorflow_models/retrain_pruned_model/")
# tfmodel = tf.keras.models.load_model('./tensorflow_models/retrain_pruned_model/')

In [89]:
# input_tensor = tf.convert_to_tensor(input_np,dtype='float32')
# print(tfmodel(input_tensor))

In [96]:
k_model = tf.keras.models.load_model("./keras_models/retrain_model.keras", safe_mode=False)

In [97]:
k_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 1, 1, 30225)]     0         
                                                                 
 LAYER_0 (Conv2D)            (None, 5, 1, 15109)       50        
                                                                 
 LAYER_1 (Activation)        (None, 5, 1, 15109)       0         
                                                                 
 LAYER_2 (Conv2D)            (None, 32, 1, 7553)       832       
                                                                 
 LAYER_3 (Activation)        (None, 32, 1, 7553)       0         
                                                                 
 LAYER_4 (MaxPooling2D)      (None, 32, 1, 151)        0         
                                                                 
 LAYER_5 (Permute)           (None, 1, 32, 151)        0     